In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf

plt.style.use("seaborn-v0_8-whitegrid")


In [ ]:
a1_df = pd.read_csv("/kaggle/input/backtest/IADI_A1.csv")
b2_df = pd.read_csv("/kaggle/input/backtest/IADI_B2.csv")
c3_df = pd.read_csv("/kaggle/input/backtest/IADI_C3.csv")

a1_tickers = a1_df['Ticker'].dropna().unique().tolist()
b2_tickers = b2_df['Ticker'].dropna().unique().tolist()
c3_tickers = c3_df['Ticker'].dropna().unique().tolist()


In [ ]:
def get_price_data(ticker_list):
    df = yf.download(ticker_list, start="2021-01-01", end="2024-01-01", auto_adjust=True)['Close']
    df = df.dropna(axis=1, thresh=int(0.75 * len(df)))  # remove tickers with excessive NaNs
    return df


In [ ]:
a1_prices = get_price_data(a1_tickers)
b2_prices = get_price_data(b2_tickers)
c3_prices = get_price_data(c3_tickers)

def build_portfolio(prices_df):
    returns = prices_df.pct_change().dropna()
    port_returns = returns.mean(axis=1)
    cum_returns = (1 + port_returns).cumprod()
    return cum_returns, port_returns

a1_cum, a1_daily = build_portfolio(a1_prices)
b2_cum, b2_daily = build_portfolio(b2_prices)
c3_cum, c3_daily = build_portfolio(c3_prices)


In [ ]:
nifty = yf.download("^NSEI", start="2021-01-01", end="2024-01-01", auto_adjust=True)['Close']
nifty_returns = nifty.pct_change().dropna()
nifty_cum = (1 + nifty_returns).cumprod()
nifty_cum = nifty_cum[nifty_cum.index.isin(a1_cum.index)]


In [ ]:
a1_tickers = ["RELIANCE.NS", "INFY.NS", "TCS.NS"]
b2_tickers = ["HDFCBANK.NS", "ITC.NS", "BAJAJFINSV.NS"]
c3_tickers = ["NTPC.NS", "COALINDIA.NS", "SBIN.NS"]



In [ ]:
def get_price_data(ticker_list):
    prices = yf.download(ticker_list, start="2021-01-01", end="2024-01-01", auto_adjust=True)['Close']
    if isinstance(prices, pd.Series):
        prices = prices.to_frame()
    prices = prices.dropna(axis=1, thresh=int(0.75 * len(prices)))
    print(f"📈 Prices shape: {prices.shape}")
    print(f"✅ Valid tickers: {list(prices.columns)}")
    return prices


In [ ]:
a1_prices = get_price_data(a1_tickers)
b2_prices = get_price_data(b2_tickers)
c3_prices = get_price_data(c3_tickers)


In [ ]:
a1_cum, a1_daily = build_portfolio(a1_prices)



In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# STEP 1: Generate quarterly timestamps
dates = pd.date_range("2021-01-01", "2024-01-01", freq="Q")
np.random.seed(42)

# STEP 2: Simulate macro indicator values
macro_df = pd.DataFrame({
    "Date": dates,
    "India_VIX": np.random.normal(16, 3, len(dates)),        # Volatility index
    "Repo_Rate": np.random.normal(5.5, 0.75, len(dates)),    # RBI policy rate
    "CPI_YoY": np.random.normal(5.8, 0.5, len(dates))        # YoY inflation %
})

# STEP 3: Classify regime per quarter
def classify_regime(row):
    if row["India_VIX"] > 18 and row["CPI_YoY"] > 6:
        return "Volatile"
    elif row["India_VIX"] < 15 and row["CPI_YoY"] < 5.5:
        return "Growth"
    else:
        return "Disinflation"

macro_df["Regime"] = macro_df.apply(classify_regime, axis=1)
macro_df.set_index("Date", inplace=True)

# STEP 4: Plot macro indicators with regime color overlays
fig, axs = plt.subplots(3, 1, figsize=(12, 9), sharex=True)

macro_df["India_VIX"].plot(ax=axs[0], color='darkorange')
axs[0].set_title("India VIX (Volatility Index)")

macro_df["Repo_Rate"].plot(ax=axs[1], color='steelblue')
axs[1].set_title("RBI Repo Rate (%)")

macro_df["CPI_YoY"].plot(ax=axs[2], color='seagreen')
axs[2].set_title("CPI YoY Inflation (%)")

for ax in axs:
    for date, row in macro_df.iterrows():
        color = {"Growth": "green", "Volatile": "red", "Disinflation": "gray"}[row["Regime"]]
        ax.axvspan(date, date + pd.Timedelta(days=90), color=color, alpha=0.15)

plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Load the synthetic price data
price_df = pd.read_csv("/kaggle/input/prices/nse200_prices.csv", parse_dates=["Date"], index_col="Date")

# Load the IADI profiles
a1_df = pd.read_csv("/kaggle/input/backtest/IADI_A1.csv")  # adjust folder name if needed
b2_df = pd.read_csv("/kaggle/input/backtest/IADI_B2.csv")
c3_df = pd.read_csv("/kaggle/input/backtest/IADI_C3.csv")

# Extract tickers from each
a1_tickers = [t for t in a1_df["Ticker"].unique() if t in price_df.columns]
b2_tickers = [t for t in b2_df["Ticker"].unique() if t in price_df.columns]
c3_tickers = [t for t in c3_df["Ticker"].unique() if t in price_df.columns]

print(f"✅ Matching tickers found:\n A1: {len(a1_tickers)} | B2: {len(b2_tickers)} | C3: {len(c3_tickers)}")

# Calculate daily returns
returns = price_df.pct_change().dropna()

# Equal-weighted portfolio returns for each profile
a1_daily = returns[a1_tickers].mean(axis=1)
b2_daily = returns[b2_tickers].mean(axis=1)
c3_daily = returns[c3_tickers].mean(axis=1)

# Cumulative returns
a1_cum = (1 + a1_daily).cumprod()
b2_cum = (1 + b2_daily).cumprod()
c3_cum = (1 + c3_daily).cumprod()

# Plot
plt.figure(figsize=(12, 6))
plt.plot(a1_cum, label='IADI A1 (Aggressive)', color='red')
plt.plot(b2_cum, label='IADI B2 (Moderate)', color='orange')
plt.plot(c3_cum, label='IADI C3 (Conservative)', color='green')
plt.title("Cumulative Return Comparison (2021–2024)")
plt.ylabel("Portfolio Value (normalized)")
plt.xlabel("Date")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# Performance Metrics
def compute_metrics(daily_returns):
    cagr = (1 + daily_returns).prod()**(252 / len(daily_returns)) - 1
    vol = daily_returns.std() * np.sqrt(252)
    sharpe = daily_returns.mean() / daily_returns.std() * np.sqrt(252)
    dd = (1 - (1 + daily_returns).cumprod() / (1 + daily_returns).cumprod().cummax()).max()
    return {
        "CAGR": round(cagr, 4),
        "Volatility": round(vol, 4),
        "Sharpe Ratio": round(sharpe, 2),
        "Max Drawdown": round(dd, 4)
    }

print("📊 IADI Portfolio Metrics")
print("A1:", compute_metrics(a1_daily))
print("B2:", compute_metrics(b2_daily))
print("C3:", compute_metrics(c3_daily))


In [ ]:
print("🧾 Columns in synthetic price data:")
print(price_df.columns.tolist())


In [ ]:
# TEMPORARY PROFILE MAPPING (synthetic tickers)
a1_tickers = [f"TICK{i}.NS" for i in range(1, 11)]     # Aggressive: TICK1 to TICK10
b2_tickers = [f"TICK{i}.NS" for i in range(11, 21)]    # Moderate: TICK11 to TICK20
c3_tickers = [f"TICK{i}.NS" for i in range(21, 31)]    # Conservative: TICK21 to TICK30

print("✅ Assigned synthetic tickers to IADI profiles.")


In [ ]:
# Daily returns
returns = price_df.pct_change().dropna()
a1_daily = returns[a1_tickers].mean(axis=1)
b2_daily = returns[b2_tickers].mean(axis=1)
c3_daily = returns[c3_tickers].mean(axis=1)

# Cumulative returns
a1_cum = (1 + a1_daily).cumprod()
b2_cum = (1 + b2_daily).cumprod()
c3_cum = (1 + c3_daily).cumprod()

# Plot
plt.figure(figsize=(12, 6))
plt.plot(a1_cum, label='IADI A1 (Aggressive)', color='red')
plt.plot(b2_cum, label='IADI B2 (Moderate)', color='orange')
plt.plot(c3_cum, label='IADI C3 (Conservative)', color='green')
plt.title("Cumulative Return Comparison (2021–2024)")
plt.ylabel("Portfolio Value (normalized)")
plt.xlabel("Date")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# Performance Metrics
def compute_metrics(daily_returns):
    cagr = (1 + daily_returns).prod()**(252 / len(daily_returns)) - 1
    vol = daily_returns.std() * np.sqrt(252)
    sharpe = daily_returns.mean() / daily_returns.std() * np.sqrt(252)
    dd = (1 - (1 + daily_returns).cumprod() / (1 + daily_returns).cumprod().cummax()).max()
    return {
        "CAGR": round(cagr, 4),
        "Volatility": round(vol, 4),
        "Sharpe Ratio": round(sharpe, 2),
        "Max Drawdown": round(dd, 4)
    }

print("📊 IADI Portfolio Metrics")
print("A1:", compute_metrics(a1_daily))
print("B2:", compute_metrics(b2_daily))
print("C3:", compute_metrics(c3_daily))
